In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_path = "microsoft/Phi-3-medium-4k-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16, trust_remote_code=True)
device = "cuda:0"
model = model.to(device)
model = model.half()
model.eval()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attenton` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 5120, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-39): 40 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear(in_features=5120, out_features=5120, bias=False)
          (qkv_proj): Linear(in_features=5120, out_features=7680, bias=False)
          (rotary_emb): Phi3RotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=5120, out_features=35840, bias=False)
          (down_proj): Linear(in_features=17920, out_features=5120, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm()
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm()
      )
    )
    (norm): Phi3RMSNorm()
  )
  (lm_head): Linear(in_features=5120, out_features=320

In [2]:
instruction = '# instruction: Write a text explaining the following fugure.'
shot_format = '''
# Information
{information}

# Answer
{answer}'''
print(shot_format)


# Information
{information}

# Answer
{answer}


In [3]:
from pathlib import Path
import json 

shots = []
shot_file = Path('preprocessed/valid/그래프 기반 문장 생성 말뭉치 2022.jsonl').open()
for shot in shot_file.readlines():
    line = json.loads(shot)
    title = line['metadata']['title']
    figure_title = line['metadata']['figure_title']
    date = line['metadata']['date']
    publisher = line['metadata']['publisher']
    figure = line['figure']
    
    reference = line['str_annotation']['reference_str']
    paraphrase = line['str_annotation']['paraphrase_str']
    
    information = {
        'title': title,
        'date': date,
        'publisher': publisher,
        'figure_title': figure_title,
        'figure': figure
    }
    shots.append(shot_format.format(information=information, answer=reference))

In [4]:
prompt = instruction + '\n'.join(shots) + '\n'
print(prompt)

# instruction: Write a text explaining the following fugure.
# Information
{'title': '수입 조제분유, 수입가격 대비 국내 판매가격은 1.8~4.1배', 'date': '2022-02-28', 'publisher': '한국소비자원', 'figure_title': '조제분유 선택 이유', 'figure': {'type': '가로막대그래프', 'x_label': '조제분유 선택 이유', 'y_label': '응답률(%, 500명)', 'x': ['아이가 잘 먹어서', '간편해서(휴대·조제)', '가격 때문에', '안전(제조과정, 유통과정)한 제품일 것 같아서', '영양성분 때문에', '모유와 가장 유사할 것 같아서', '원유가 좋을 것 같아서', '기타'], 'y': [44.4, 4.8, 4, 9.6, 20.6, 6.8, 4.8, 5]}}

# Answer
이 그래프는 가장 최근 구매한 조제분유 선택 이유에 대해 설문한 것이다. '아이가 잘 먹어서' 해당 조제 분유를 구매하였다는 응답이 44.4%로 가장 높았고, '영양성분'(20.6%), '안전한 제품'(9.6%) 등의 응답이 뒤를 이었다. 반면에 '가격 때문에' 해당 조제 분유를 구매하였다는 응답이 4.0%로 가장 낮았고, '간편해서'(4.8%), '원유가 좋을 것 같아서'(4.8%) 등의 응답이 뒤를 이었다. 수입 조제분유가 수입 가격 대비 국내 판매가격이 1.8~4.1배나 높지만 가격에 의해 구매를 결정하는 소비자가 가장 적은 것을 확인할 수 있다.

# Information
{'title': '교육 분야 5년(2017~2022) 성과자료집 발간', 'date': '2022-04-21', 'publisher': '교육부', 'figure_title': '고교 무상교육 지원 인원', 'figure': {'type': '세로막대그래프', 'x_label': '연도', 'y_label': '지원 인원(만 명)', 'x': [2019, 2020, 2

In [5]:
from pathlib import Path
import json
import re
from tqdm import tqdm

settings = json.loads(Path('../../../settings.json').read_text())preprocessed_data_path = Path(settings['preprocessed_data_path'])data_path = Path('.').resolve()data_name = data_path.namesource_name = data_path.parent.namepreprocessed_dir = preprocessed_data_path/source_name/data_namesource_data_dir = preprocessed_dir/'preprocessed'
splits = ['train', 'valid']
tasks = ['그래프 설명 생성']
task_data_dir = preprocessed_dir/'preprocessed_task'
task_data_dir.mkdir(exist_ok=True)
for task in tasks:
    task_path = task_data_dir/task
    task_path.mkdir(exist_ok=True)
    
#### prepare for task preprocess



#### prepare for task preprocess end

#### task preprocess
for split in splits:
    source_data_dir_split = source_data_dir/split
    task_files = [(task_data_dir/task/f'{split}.jsonl').open('w', encoding='utf-8') for task in tasks]
    for source_data in tqdm(list(source_data_dir_split.iterdir()), desc=split):
        source_data = source_data.open()
        for line in source_data.readlines():
            line = json.loads(line)
            
            #### data preprocess
            title = line['metadata']['title']
            figure_title = line['metadata']['figure_title']
            date = line['metadata']['date']
            publisher = line['metadata']['publisher']
            figure = line['figure']
            
            reference = line['str_annotation']['reference_str']
            paraphrase = line['str_annotation']['paraphrase_str']
            
            information = {
                'title': title,
                'date': date,
                'publisher': publisher,
                'figure_title': figure_title,
                'figure': figure
            }
            prompt_text = prompt+shot_format.format(information=information, answer='')
            
            inputs = tokenizer.encode(prompt_text, return_tensors="pt").to(device)
            with torch.no_grad():
                output = model.generate(
                    inputs,
                    max_new_tokens=500, 
                    num_return_sequences=1,
                    do_sample=True,
                    top_k=50,
                    top_p=0.95,
                    temperature=0.7
                )
            generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
            response = generated_text[len(prompt_text):].split('# Information')[0].strip()
            #### data preprocess end 


            #### 그래프 설명 생성
            data = {'input': None, 'output': None}
            ## preprocess data from line
            data['input'] = information
            data['output'] = [reference, paraphrase, response]
            ## preprocess data from line end
            task_files[0].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 그래프 설명 생성 end

            
    #         break
    #     break
    # break

    for path in task_files:
        path.close()      
        
#### task preprocess end      

valid: 100%|██████████| 1/1 [00:35<00:00, 35.83s/it]


: 